# © Artur Czarnecki. All rights reserved.
# Intergrax framework – proprietary and confidential.
# Use, modification, or distribution without written permission is prohibited.

In [1]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

Basic sanity-check notebook for Drop-In Knowledge Mode runtime.

Goals:
  - Verify that DropInKnowledgeRuntime can:
      * create or load a session,
      * append user and assistant messages,
      * build conversation history from SessionStore,
      * return a RuntimeAnswer object.
  - Use InMemorySessionStore for simplicity.
  - Keep LLM integration as placeholder for now.

In [2]:
from datetime import datetime
from pprint import pprint

from intergrax.runtime.drop_in_knowledge_mode.config import RuntimeConfig
from intergrax.runtime.drop_in_knowledge_mode.engine import DropInKnowledgeRuntime
from intergrax.runtime.drop_in_knowledge_mode.response_schema import RuntimeRequest
from intergrax.runtime.drop_in_knowledge_mode.session_store import (
    InMemorySessionStore,
)

c:\ProgramData\anaconda3\envs\genai_longchain\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Create config, session store, runtime

In [3]:
from langchain_ollama import ChatOllama
from intergrax.llm_adapters import LangChainOllamaAdapter

# Instantiate the in-memory session store
session_store = InMemorySessionStore()

# Instantiate a LLM adapter
llm_adapter = LangChainOllamaAdapter(
    chat = ChatOllama(
        model = "llama3.1:latest"
    )
)

# For now we do not need real embedding/vectorstore managers for the skeleton;
# if your RuntimeConfig requires concrete instances at import time, you can
# inject the ones you already use in your RAG pipelines.
from intergrax.rag.embedding_manager import IntergraxEmbeddingManager
from intergrax.rag.vectorstore_manager import IntergraxVectorstoreManager, VSConfig

# Example minimal initialization; adjust to your actual constructors/configs.
embedding_manager = IntergraxEmbeddingManager(    
    provider="ollama",
    model_name="rjmalagon/gte-qwen2-1.5b-instruct-embed-f16:latest", 
    assume_ollama_dim=1536)


TENANT = "intergrax"
CORPUS = "intergrax-strategy"
VERSION = "v1"

# Initialize the vector store at startup (without loading/splitting/embedding yet).
# This gives us a handle to the underlying database and allows for a light-weight
# presence check before performing any expensive ingestion work.
vectorstore_cfg = VSConfig(
    provider="chroma",
    collection_name="intergrax_docs"
)

vectorstore_manager = IntergraxVectorstoreManager(config=vectorstore_cfg, verbose=True)

config = RuntimeConfig(
    llm_adapter=llm_adapter,
    embedding_manager=embedding_manager,
    vectorstore_manager=vectorstore_manager,
    llm_label="llm-adapter",
    embedding_label="default-embedding",
    vectorstore_label="default-vectorstore",
    enable_rag=False,        # disabled for this initial sanity check
    enable_websearch=False,  # disabled for this initial sanity check
    enable_tools=False,      # disabled for this initial sanity check
)

runtime = DropInKnowledgeRuntime(
    config=config,
    session_store=session_store,
)

print("Runtime initialized at", datetime.utcnow().isoformat())


[intergraxVectorstoreManager] Initialized provider=chroma, collection=intergrax_docs
[intergraxVectorstoreManager] Existing count: 0
Runtime initialized at 2025-11-21T11:24:54.596237


C:\Users\XPS\AppData\Local\Temp\ipykernel_15552\2928511357.py:58: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  print("Runtime initialized at", datetime.utcnow().isoformat())


First request: create a new session

In [4]:
# Define user and session identifiers
user_id = "demo-user-001"
session_id = "demo-session-001"

request_1 = RuntimeRequest(
    user_id=user_id,
    session_id=session_id,
    message="Hello Intergrax runtime, this is my first message.",
    metadata={"source": "notebook-demo"},
)

answer_1 = await runtime.ask(request_1)

print("=== ANSWER 1 ===")
print(answer_1.answer)
print("\nRoute info:")
pprint(answer_1.route)
print("\nStats:")
pprint(answer_1.stats)
print("\nDebug trace:")
pprint(answer_1.debug_trace)


=== ANSWER 1 ===
Welcome to our Intergrax runtime environment.

What brings you here today? Do you have any specific tasks or projects in mind that you'd like to work on, or would you like some suggestions on how to get started with the Intergrax platform?

Route info:
RouteInfo(used_rag=False,
          used_websearch=False,
          used_tools=False,
          used_long_term_memory=False,
          used_user_profile=True,
          strategy='simple_placeholder',
          extra={})

Stats:
RuntimeStats(total_tokens=None,
             input_tokens=None,
             output_tokens=None,
             rag_tokens=None,
             websearch_tokens=None,
             tool_tokens=None,
             duration_ms=None,
             extra={})

Debug trace:
{'config': {'embedding_label': 'default-embedding',
            'llm_label': 'llm-adapter',
            'vectorstore_label': 'default-vectorstore'},
 'history_length': 1,
 'session_id': 'demo-session-001',
 'user_id': 'demo-user-001'}


Inspect session state after first request

In [5]:
session = await session_store.get_session(answer_1.debug_trace["session_id"])

print("=== SESSION AFTER FIRST REQUEST ===")
print("Session ID:", session.id)
print("User ID:", session.user_id)
print("Messages count:", len(session.messages))
print("Created at:", session.created_at)
print("Updated at:", session.updated_at)

print("\nMessages:")
for idx, msg in enumerate(session.messages, start=1):
    print(f"  [{idx}] role={msg.role!r}, created_at={msg.created_at.isoformat()}")
    print(f"      content={msg.content!r}")


=== SESSION AFTER FIRST REQUEST ===
Session ID: demo-session-001
User ID: demo-user-001
Messages count: 2
Created at: 2025-11-21 11:24:54.601460+00:00
Updated at: 2025-11-21 11:24:59.433174+00:00

Messages:
  [1] role='user', created_at=2025-11-21T11:24:54.601460+00:00
      content='Hello Intergrax runtime, this is my first message.'
  [2] role='assistant', created_at=2025-11-21T11:24:59.433174+00:00
      content="Welcome to our Intergrax runtime environment.\n\nWhat brings you here today? Do you have any specific tasks or projects in mind that you'd like to work on, or would you like some suggestions on how to get started with the Intergrax platform?"


Second request: same session, history should grow

In [6]:
request_2 = RuntimeRequest(
    user_id=user_id,
    session_id=session_id,
    message="This is my second message in the same session.",
    metadata={"source": "notebook-demo"},
)

answer_2 = await runtime.ask(request_2)

print("=== ANSWER 2 ===")
print(answer_2.answer)
print("\nRoute info:")
pprint(answer_2.route)
print("\nDebug trace:")
pprint(answer_2.debug_trace)

session_2 = await session_store.get_session(answer_2.debug_trace["session_id"])
print("\n=== SESSION AFTER SECOND REQUEST ===")
print("Session ID:", session_2.id)
print("Messages count:", len(session_2.messages))
for idx, msg in enumerate(session_2.messages, start=1):
    print(f"  [{idx}] role={msg.role!r}, created_at={msg.created_at.isoformat()}")
    print(f"      content={msg.content!r}")


=== ANSWER 2 ===
So we're still going strong after your first message! Just for clarity, I should mention that this conversation just started, and there's no prior context or messages to recall from a previous session. But feel free to start fresh and ask me anything you'd like to know about Intergrax runtime or any other topic that interests you!

Route info:
RouteInfo(used_rag=False,
          used_websearch=False,
          used_tools=False,
          used_long_term_memory=False,
          used_user_profile=True,
          strategy='simple_placeholder',
          extra={})

Debug trace:
{'config': {'embedding_label': 'default-embedding',
            'llm_label': 'llm-adapter',
            'vectorstore_label': 'default-vectorstore'},
 'history_length': 3,
 'session_id': 'demo-session-001',
 'user_id': 'demo-user-001'}

=== SESSION AFTER SECOND REQUEST ===
Session ID: demo-session-001
Messages count: 4
  [1] role='user', created_at=2025-11-21T11:24:54.601460+00:00
      content='Hello

Verify that history length is limited by config

In [ ]:
from intergrax.llm.messages import ChatMessage

# For debugging: call the history builder directly
session_for_history = await session_store.get_session(answer_2.debug_trace["session_id"])
history = runtime._build_chat_history(session_for_history)

print("=== BUILT CHAT HISTORY ===")
print("History length:", len(history))
for idx, msg in enumerate(history, start=1):
    assert isinstance(msg, ChatMessage)
    print(f"  [{idx}] role={msg.role!r}, created_at={msg.created_at}")
    print(f"      content={msg.content!r}")


=== BUILT CHAT HISTORY ===
History length: 4
  [1] role='user', created_at=2025-11-21T11:24:54.601460+00:00
      content='Hello Intergrax runtime, this is my first message.'
  [2] role='assistant', created_at=2025-11-21T11:24:59.433174+00:00
      content="Welcome to our Intergrax runtime environment.\n\nWhat brings you here today? Do you have any specific tasks or projects in mind that you'd like to work on, or would you like some suggestions on how to get started with the Intergrax platform?"
  [3] role='user', created_at=2025-11-21T11:24:59.446306+00:00
      content='This is my second message in the same session.'
  [4] role='assistant', created_at=2025-11-21T11:25:01.244703+00:00
      content="So we're still going strong after your first message! Just for clarity, I should mention that this conversation just started, and there's no prior context or messages to recall from a previous session. But feel free to start fresh and ask me anything you'd like to know about Intergrax runt